In [ ]:
import os
import requests
import zipfile
import shutil

# URLs for datasets
urls = {
    "DB1": "http://bias.csr.unibo.it/fvc2004/Downloads/DB1_B.zip",
    "DB2": "http://bias.csr.unibo.it/fvc2004/Downloads/DB2_B.zip",
    "DB3": "http://bias.csr.unibo.it/fvc2004/Downloads/DB3_B.zip",
    "DB4": "http://bias.csr.unibo.it/fvc2004/Downloads/DB4_B.zip"
}

# Directory to save datasets
base_dir = "/content/fingerprint_datasets"
final_dir = "/content/DBs_Dataset"
original_images_dir = os.path.join(final_dir, "original_images")

# Create the necessary directories
os.makedirs(original_images_dir, exist_ok=True)

# Function to download and extract dataset
def download_and_extract(url, save_dir):
    # Create a folder for each database to store the extracted files separately
    db_name = os.path.splitext(os.path.basename(url))[0]
    db_save_dir = os.path.join(save_dir, db_name)
    os.makedirs(db_save_dir, exist_ok=True)

    zip_path = os.path.join(db_save_dir, os.path.basename(url))

    if not os.path.exists(zip_path):
        response = requests.get(url)
        with open(zip_path, 'wb') as f:
            f.write(response.content)

    # Extract the zip file to the corresponding directory
    extract_dir = os.path.splitext(zip_path)[0]
    if not os.path.exists(extract_dir):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)

    return extract_dir, db_name

# Download and extract all datasets into separate directories
extracted_dirs = {}
for db, url in urls.items():
    extracted_dirs[db] = download_and_extract(url, base_dir)

# Move and rename all image files into the final folder with prefix
for db, (db_dir, db_name) in extracted_dirs.items():
    for root, dirs, files in os.walk(db_dir):
        for file in files:
            if file.endswith('.tif'):
                # Construct the new file name with "FVC2004_" prefix
                new_name = f"FVC2004_{db_name}_{file}"  # Add "FVC2004_" prefix here
                old_path = os.path.join(root, file)
                new_path = os.path.join(original_images_dir, new_name)

                # Rename and move the file
                shutil.move(old_path, new_path)

# Delete the 'fingerprint_datasets' directory after the work is complete
shutil.rmtree(base_dir)

# Final output print
print(f"All images are renamed and moved to {original_images_dir}, and {base_dir} has been deleted.")


All images are renamed and moved to /content/DBs_Dataset/original_images, and /content/fingerprint_datasets has been deleted.


In [ ]:
!git clone https://github.com/raffaele-cappelli/pyfing.git


Cloning into 'pyfing'...
remote: Enumerating objects: 196, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 196 (delta 24), reused 28 (delta 13), pack-reused 142 (from 1)
Receiving objects: 100% (196/196), 104.90 MiB | 33.70 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [ ]:
# Define the path to the file
file_path = '/content/pyfing/pyfing/segmentation.py'

# Open the file in read mode to get its content
with open(file_path, 'r') as file:
    content = file.readlines()

# Modify the specific line in the content
modified_content = []
for line in content:
    if "from .definitions import Image, Parameters" in line:
        modified_content.append("from definitions import Image, Parameters\n")
    else:
        modified_content.append(line)

# Write the modified content back to the file
with open(file_path, 'w') as file:
    file.writelines(modified_content)

print(f"Modified the file {file_path} successfully.")


Modified the file /content/pyfing/pyfing/segmentation.py successfully.


In [ ]:
import requests
import zipfile
import os
import shutil
import re

# URL to the dataset
url = 'https://figshare.com/ndownloader/files/1871964'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Save the zip file to disk
    zip_file_path = '/content/dataset_file.zip'
    with open(zip_file_path, 'wb') as file:
        file.write(response.content)
    print("File downloaded successfully.")

    # Create the target folder if it doesn't exist
    target_folder = '/content/DBs_Dataset/validation_images'
    os.makedirs(target_folder, exist_ok=True)

    # Extract the zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Extract all files to the '/content/dataset' folder
        zip_ref.extractall('/content/dataset')
        print("Files extracted successfully.")

        # Define the correct path to the 'GroundTruth' folder
        extracted_folder = '/content/dataset/SegmentationBenchmark/GroundTruth'

        # Check if the 'GroundTruth' folder exists
        if os.path.exists(extracted_folder):
            # Loop through all the files in the 'GroundTruth' folder
            for file_name in os.listdir(extracted_folder):
                file_path = os.path.join(extracted_folder, file_name)

                # Check if the file is an image (you can add more extensions as needed)
                if file_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
                    # Updated regular expression to match filenames like FVC2000_DB1_IM_54_6seg.png
                    match = re.match(r"([A-Za-z0-9]+)_IM_(\d+_\d+)[^.]+(\.png|\.jpg|\.jpeg|\.bmp|\.gif)$", file_name)
                    if match:
                        # Extract the parts of the filename for renaming
                        part1 = match.group(1)   # This captures the first part (e.g., FVC2000_DB1)
                        part2 = match.group(2)   # This captures the second part (e.g., 54_6)

                        # New file name with the '_B_' in place of 'IM' and '.tif' extension
                        new_file_name = f"{part1}_B_{part2}.tif"
                        new_file_path = os.path.join(target_folder, new_file_name)

                        # Move and rename the file
                        shutil.move(file_path, new_file_path)
                        #print(f"Renamed and moved: {file_name} to {new_file_name}")
                    else:
                        # If the file doesn't match the expected pattern, move it without renaming
                        shutil.move(file_path, os.path.join(target_folder, file_name))
                        #print(f"Moved: {file_name} without renaming.")



File downloaded successfully.
Files extracted successfully.


In [ ]:
import os
from PIL import Image

# Directory containing the images
input_directory = '/content/DBs_Dataset/validation_images/'

# Iterate through all the files in the directory
for filename in os.listdir(input_directory):
    # Check if the file is a PNG image (you can change to match other formats as well)
    if filename.endswith('.png'):
        # Construct the full file path
        file_path = os.path.join(input_directory, filename)

        # Check if the file contains "IM" and rename it to "B"
        if "IM" in filename:
            new_filename = filename.replace("IM", "B")
            new_filename = new_filename.replace(".png", ".tif")  # Change extension to .tif

            # Construct the new file path
            new_file_path = os.path.join(input_directory, new_filename)

            # Open the image using PIL
            with Image.open(file_path) as img:
                # Convert image to grayscale
                img_gray = img.convert('L')

                # Apply a binary threshold (you can adjust the threshold value)
                threshold = 128  # This can be adjusted
                img_binary = img_gray.point(lambda p: p > threshold and 255)

                # Save the binary image in TIFF format
                img_binary.save(new_file_path, format='TIFF')

            # Optionally, delete the original image (uncomment if you want to remove it)
            # os.remove(file_path)

            # Print the renaming action
            #print(f"Renamed and converted {filename} to {new_filename}")

shutil.rmtree('/content/dataset')

import os

# Specify the file path
file_path = '/content/dataset_file.zip'

# Remove the file
os.remove(file_path)

# Confirm deletion
print(f"{file_path} has been deleted.")


/content/dataset_file.zip has been deleted.


In [ ]:
import cv2 as cv
import os
from glob import glob

import sys
sys.path.append('/content/pyfing/pyfing')

# Now you should be able to import the segmentation module
import segmentation
from segmentation import Gmfs, GmfsParameters

# Directories
input_dir = '/content/DBs_Dataset/original_images'
output_dir = '/content/DBs_Dataset/segmented_images'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Loop through all .tif images in the original_images directory
for image_path in glob(os.path.join(input_dir, '*.tif')):
    # Read the image in grayscale
    image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)

    # Check if the image was loaded successfully
    if image is not None:
        # Initialize the GmfsParameters and Gmfs objects
        parameters = GmfsParameters()
        segmentation_method = Gmfs(parameters)

        # Perform the segmentation
        segmented_image = segmentation_method.run(image)

        # Construct the output path with the original name and "_seg" suffix
        filename = os.path.basename(image_path)
        output_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_paperseg.tif")

        # Save the segmented image
        cv.imwrite(output_path, segmented_image)

# Final output print
print(f"Segmentation completed. Segmented images are saved in {output_dir}.")


Segmentation completed. Segmented images are saved in /content/DBs_Dataset/segmented_images.


In [ ]:
import cv2
import numpy as np
import os

# Path to the dataset folder
dataset_folder = '/content/DBs_Dataset/segmented_images'

# Function to check the center of an image and process accordingly
def process_image(image_path):
    # Load the image in grayscale mode
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    if img is None:
        print(f"Error loading image {image_path}")
        return

    # Get the center pixel coordinates
    height, width = img.shape
    center = (height // 2, width // 2)

    # Get the center pixel value (0 for black, 255 for white in grayscale)
    center_pixel = img[center]

    if center_pixel == 255:  # Black center, invert the image
        inverted_img = cv2.bitwise_not(img)
        # Overwrite the original image with the inverted image
        cv2.imwrite(image_path, inverted_img)


# Iterate through all images in the dataset folder
for filename in os.listdir(dataset_folder):
    image_path = os.path.join(dataset_folder, filename)

    # Check if it's an image file (you can add more formats if needed)
    if filename.endswith(('.jpg', '.png', '.tif')):
        process_image(image_path)


In [ ]:
import cv2
import numpy as np
import os
from glob import glob

# Pre-processing and error calculation
def calculate_error_probabilities(seg_image_path, val_image_path):
    # Load the segmented image and the validation image (ground truth)
    seg_image = cv2.imread(seg_image_path, cv2.IMREAD_GRAYSCALE)
    val_image = cv2.imread(val_image_path, cv2.IMREAD_GRAYSCALE)

    if seg_image is None or val_image is None:
        print(f"Error: Could not load images {seg_image_path} or {val_image_path}")
        return None, None, None  # Return None if there is an issue loading images

    # Ensure the images are binary (foreground=255, background=0)
    _, seg_image = cv2.threshold(seg_image, 128, 255, cv2.THRESH_BINARY)
    _, val_image = cv2.threshold(val_image, 128, 255, cv2.THRESH_BINARY)

    # Calculate True Background (Nbrb) and True Foreground (Nbrf)
    Nbrb = np.sum(val_image == 0)  # True background pixels in validation
    Nbrf = np.sum(val_image == 255)  # True foreground pixels in validation

    # Calculate Number of Background Classification Errors (Nbrbe)
    Nbrbe = np.sum((seg_image == 255) & (val_image == 0))  # Foreground classified as background

    # Calculate Number of Foreground Classification Errors (Nbrfe)
    Nbrfe = np.sum((seg_image == 0) & (val_image == 255))  # Background classified as foreground

    # Calculate Prob1 and Prob2
    Prob1 = Nbrbe / Nbrb if Nbrb > 0 else 0  # Probability that foreground is classified as background
    Prob2 = Nbrfe / Nbrf if Nbrf > 0 else 0  # Probability that background is classified as foreground

    # Calculate ProbErr
    ProbErr = (Prob1 + Prob2) / 2

    return Prob1, Prob2, ProbErr

# Directories
segmentation_dir = '/content/DBs_Dataset/segmented_images'
validation_dir = '/content/DBs_Dataset/validation_images'

# List of image prefixes for DB1, DB2, DB3, DB4
datasets = ['DB1', 'DB2', 'DB3', 'DB4']

# Initialize a dictionary to store the results
final_results = {}

# Calculate the error probabilities for each image in the segmentation folder
for dataset in datasets:
    # Store the results for the current dataset
    Prob1_values = []
    Prob2_values = []
    ProbErr_values = []

    # Loop through each image in the dataset's segmentation folder
    for image_path in glob(os.path.join(segmentation_dir, f'FVC2004_{dataset}_*.tif')):
        # Construct the corresponding validation image path
        val_image_name = os.path.basename(image_path).replace('_paperseg.tif', 'seg.tif')
        val_image_path = os.path.join(validation_dir, val_image_name)

        # Calculate the error probabilities
        Prob1, Prob2, ProbErr = calculate_error_probabilities(image_path, val_image_path)

        if Prob1 is not None:
            # Store the values in the corresponding lists
            Prob1_values.append(Prob1)
            Prob2_values.append(Prob2)
            ProbErr_values.append(ProbErr)

    # Calculate the average for each probability
    if Prob1_values:
        avg_Prob1 = np.mean(Prob1_values)
        avg_Prob2 = np.mean(Prob2_values)
        avg_ProbErr = np.mean(ProbErr_values)

        # Store the averages for the dataset
        final_results[dataset] = {
            'avg_Prob1': avg_Prob1,
            'avg_Prob2': avg_Prob2,
            'avg_ProbErr': avg_ProbErr
        }

# Print the final average results for each dataset
for dataset, metrics in final_results.items():
    print(f"Average Results for {dataset}:")
    print(f"avg_Prob1: {metrics['avg_Prob1']:.4f}, avg_Prob2: {metrics['avg_Prob2']:.4f}, avg_ProbErr: {metrics['avg_ProbErr']:.4f}")

# Calculate and print overall averages across all datasets
all_Prob1 = [metrics['avg_Prob1'] for metrics in final_results.values()]
all_Prob2 = [metrics['avg_Prob2'] for metrics in final_results.values()]
all_ProbErr = [metrics['avg_ProbErr'] for metrics in final_results.values()]

print("\nOverall Average Results:")
print(f"Overall avg_Prob1: {np.mean(all_Prob1):.4f}, Overall avg_Prob2: {np.mean(all_Prob2):.4f}, Overall avg_ProbErr: {np.mean(all_ProbErr):.4f}")


Average Results for DB1:
avg_Prob1: 0.1054, avg_Prob2: 0.0147, avg_ProbErr: 0.0601
Average Results for DB2:
avg_Prob1: 0.0596, avg_Prob2: 0.0507, avg_ProbErr: 0.0552
Average Results for DB3:
avg_Prob1: 0.0611, avg_Prob2: 0.0066, avg_ProbErr: 0.0338
Average Results for DB4:
avg_Prob1: 0.0662, avg_Prob2: 0.0023, avg_ProbErr: 0.0342

Overall Average Results:
Overall avg_Prob1: 0.0731, Overall avg_Prob2: 0.0186, Overall avg_ProbErr: 0.0458
